# MassBalanceMachine Data Processing - Example for Retrieving Training Features for the Norway Region (WGMS)

In this notebook, we will demonstrate the data processing workflow of the MassBalanceMachine using an example with glaciological data from Norwegian glaciers. This example will guide you through the data processing pipeline, which retrieves topographical and meteorological features for all points with glaciological surface mass balance observations.

We begin by importing some basic libraries along with the ```massbalancemachine``` library. Next, we specify the location where we will store the files for the region of interest (in this case, Norway). The data used in this example is from the [WGMS database](https://wgms.ch/data_databaseversions/), and we will utilize the specified columns.

**Note:** It's important to note that WGMS data can contain errors, please check the data for correctness before using it.    

**Note:** All data, stake measurements and glaciers outlines, are expected to have the WGS84 CRS.

In [ ]:
import pandas as pd
import geopandas as gpd

import massbalancemachine as mbm

## 1. Load your Target Surface Mass Balance Dataset and Retrieve RGI ID per Stake

In this step, we define and load our glaciological data from a region of interest. The WGMS dataset does not include RGI IDs by default, so we need to retrieve them from a glacier outline shapefile provided by the Randolph Glacier Inventory (v6). Each stake is then matched with an RGI ID. The RGI ID is necessary for the MassBalanceMachine to add additional topographical and meteorological features for training stage.

**How to Retrieve the Glacier Outlines:** Download the shapefiles for the region of interest from this [link](https://daacdata.apps.nsidc.org/pub/DATASETS/nsidc0770_rgi_v6/). Extract the files and copy the .shp, .prj, .shx, and .dbf files in the correct directory so that you can use it with the Jupyter Notebook. Also, make sure you point to the correct directory and files in the next code cell.

**Note:** Data records that have an invalid FROM_DATE or TO_DATE, where the day is indicated as 99, are deleted from the dataset.

In [ ]:
# Specify the filename of the input file with the raw data
target_data_fname = './example_data/norway/files/norway_wgms_dataset.csv'
# Specify the shape filename of the glaciers outline obtained from RGIv6
glacier_outline_fname = './example_data/norway/glacier_outlines/08_rgi60_Scandinavia.shp'

# Load the target data and the glacier outlines
data = pd.read_csv(target_data_fname)
glacier_outline = gpd.read_file(glacier_outline_fname)

### 1.1 Match the Stake Measurements with a RGI ID

Based on the location of the stake measurement given by POINT_LAT and POINT_LON, each data record is matched with the RGI ID for the glacier where the stake is located.

In [ ]:
# Get the RGI ID for each stake measurement for the region of interest
data = mbm.utils.get_rgi(data, glacier_outline)

Then, we can create a MassBalanceMachine `Dataset`, by using the loaded dataframe for Norway stake data together with the matched RGI IDs, as such: 

In [ ]:
# Provide the column name for the column that has the RGI IDs for each of the stakes
dataset = mbm.Dataset(data=data, data_path='./example_data/norway/files/')

## 2. Get Topographical Features per Stake Measurement

Once we have created a `Dataset`, the first thing we can do is to add topographical data in our dataset. This can be done automatically with the MassBalanceMachine (which calls OGGM) by doing the following:

In [ ]:
# Specify the topographical features of interest 
# Please see the OGGM documentation what variables are available: https://oggm.org/tutorials/stable/notebooks/10minutes/machine_learning.html ('topo', 'slope_factor', 'dis_from_border')
voi_topographical = ['aspect', 'slope']

# Retrieve the topographical features for each of the stake measurement and add them to the dataset
dataset.get_topo_features(voi_topographical)

## 3. Get Meteorological Features per Stake Measurement

Once we have the topographical data, we can add the necessary climate data for the dataset. This is done by pulling that from ERA5-Land database for the region of interest. Before the climate data is matched with the stake measurements, we need to manually download the climate data for the region of interest from [climate Copernicus website](https://cds.climate.copernicus.eu/cdsapp#!/dataset/reanalysis-era5-land-monthly-means?tab=form). Check the following options:

- **Product type:** _Monthly averaged reanalysis_
- **Variable**:
    - _2m temperature_ (t2m)
    - _total precipitation_ (tp)
    - _surface latent heat flux_ (slhf)  
    - _surface sensible heat flux_ (sshf)
    - _surface solar radiation downwards_ (ssrd)
    - _forecast albedo_ (fal)
    - _surface net thermal radiation_ (str)
    - You can explore additional options as you prefer. In this example, we use the variables listed above.
- **Year**: _Select all_
- **Month**: _Select all_
- **Time**: _Select all_
- **Geographical area**: 
- **Format:** _NetCDF-3_

Then click _Submit Request_ (after you have registered or logged into your account). Please be aware that downloading this data can take up to several hours, depending on the number of variables, timescales, and the area selected. Once the download is complete, place the netCDF file in the correct directory and rename it accordingly.

Additionally, we need the _geopotential height_ as an extra variable in our dataset. We will calculate a new variable by determining the difference between the geopotential height and the elevation at the stake measurement. The purpose of this height difference is to encourage the model to use it for downscaling the ERA5Land climate data, rather than relying on the lapse rate. The geopotential is downloaded from [here](https://confluence.ecmwf.int/display/CKB/ERA5-Land%3A+data+documentation#ERA5Land:datadocumentation-parameterlistingParameterlistings), and is already included in this example.   

In [ ]:
# Specify the files of the climate data, that will be matched with the coordinates of the stake data
era5_climate_data = './example_data/norway/climate/era5_monthly_averaged_data.nc'
geopotential_data = './example_data/norway/climate/era5_geopotential_pressure.nc'

# Match the climate features, from the ERA5Land netCDF file, for each of the stake measurement dataset
dataset.get_climate_features(era5_climate_data, geopotential_data)

## 4. Transform Data to Monthly Resolution

Finally, we need to transform the dataset into a monthly resolution, accounting for a variable period for the SMB target data. This will be done in order to perform SMB predictions at a monthly time step, which then will be integrated both in time and space to match the available glaciological and geodetic SMB observations for the training. Please specify the climate variables used in the previous step in the list below. Consult the [documentation](https://confluence.ecmwf.int/display/CKB/ERA5-Land%3A+data+documentation) of all short names of the climate variables.

In [ ]:
voi_climate = ['t2m', 'tp', 'slhf', 'sshf', 'ssrd', 'fal', 'srt']

dataset.convert_to_monthly(voi_climate, voi_topographical)

Finally, we can take a look at the dataset which will be used for training.

In [ ]:
display(dataset.data)